In [1]:
# Ingest the index
from llama_index import StorageContext, load_index_from_storage
import os

# import langchain 
#langchain.debug = True

In [2]:
project_path="./bioe80"
storage_path=project_path+"/storage"

if os.path.isfile(project_path+"/storage/index_store.json") :
    print("Found index - loading - this may take a few seconds")
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir=storage_path)
    # load index
    index = load_index_from_storage(storage_context)
else :
	print("please generate an index first")

print(index)

Found index - loading - this may take a few seconds


In [3]:
from langchain.agents import Tool

tools = [
    Tool(
        name="BITS",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about your class. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
]

In [4]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType

# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(temperature=0, max_tokens=512, model_name="gpt-3.5-turbo") # gpt-4
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, memory=memory, verbose=True)

In [5]:
agent_chain.run(input="Hi, I am Drew")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Hello Drew! How can I assist you today?

> Finished chain.


'Hello Drew! How can I assist you today?'

In [6]:
cross_query_str = "Please summarize the BioE80 course"
response = agent_chain.run("Please summarize the BioE80 course")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: BITS
Action Input: Summarize the BioE80 course
Observation: 
BioE80 is an introductory course to bioengineering, which focuses on the systematic engineering of living systems to benefit people and the planet. The course aims to help students learn ways of thinking about engineering living matter, explore and do bioengineering starting from DNA, become more capable of learning and explaining bioengineering, and devise and express wishes for bioengineering that could be made true by 2030.


> Finished chain.


In [7]:
cross_query_str = "Will Bioe80 be offered again next year?"
response = agent_chain.run(input=cross_query_str)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: BITS
Action Input: Will Bioe80 be offered again next year?
Observation: 
It is not possible to answer this question with the given context information.


> Finished chain.


In [8]:
# demonstrate memory
agent_chain.run(input="What's my name?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: BITS
Action Input: What's my name?
Observation: 
Your name is Ahmed Best.


> Finished chain.


'\nYour name is Ahmed Best.'

In [11]:
# reinitialize agent
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(temperature=0, max_tokens=512, model_name="gpt-3.5-turbo") # gpt-4
agent_chain = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, memory=memory, verbose=True)

In [ ]:
while True:
    text_input = input("User: ")
    response = agent_chain.run(input=text_input)
    print(f"Agent: {response}")

User:  How much data can be encoded in DNA?




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: BITS
Action Input: How much data can be encoded in DNA?
Observation: 
DNA can be used to encode up to 200 megabytes of data.


> Finished chain.
Agent: 
DNA can be used to encode up to 200 megabytes of data.
